In [4]:
import requests
import json
import os

In [5]:
map_url = "https://grammanchitra.gov.in/#"

In [6]:
config_url = "https://grammanchitra.gov.in/js/config.js"

In [7]:
r = requests.get(config_url)

In [8]:
token = r.text.split('dlayertoken = "')[1].split('"')[0].strip()

In [9]:
token

'Swk5CQCB-pnToLpxQlCjkPshu0eLdlAEE6dHaV7CeMT--7V02xPX0kvbHJG-1iaS59xv8Vev63mhjQ-nsmpYTQ..'

In [10]:
mod_headers = {
    "referer": map_url,
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36 Edg/86.0.622.63"
}

In [11]:
layers = [
    "https://mapservice.gov.in/mapserviceserv176/rest/services/Panchayat/panchayat_admin/MapServer"
]

In [12]:
params = {
    "Token": token,
    "f": "pjson"
}

In [13]:
max_record_count_mapping = {
    0: 1,
    1: 50,
    2: 100,
    3: 300,
    7: 300
}

In [14]:
output_folder = "output"

In [15]:
def make_folder(folderpath):
    if folderpath == "":
        return
    if not os.path.exists(os.path.dirname(folderpath)):
        make_folder(os.path.dirname(folderpath))
    if not os.path.exists(folderpath):
        os.mkdir(folderpath)

In [16]:
def scrape(layer_url):
    metadata = requests.get(layer_url, params=params, verify=False, headers=mod_headers).json()
#     print(metadata)
    # Make Folder
    folderpath  = os.path.join(os.path.abspath(output_folder), 'scrape', metadata['name'])
    make_folder(folderpath)
    # Scrape Now
    max_record_count = metadata['maxRecordCount']
    _max_record_count = max_record_count_mapping.get(metadata['id'], max_record_count)
    print("-- Using max_record_count: ", _max_record_count)
    gparams = {
        "Token": token,
        "f": "json",
        "where": "1=1",
        "returnGeometry": 'true',
        "returnTrueCurves": 'true',
        "spatialRel": "",
        "geometry": "",
        "geometryType": "",
        "esriGeometryEnvelope": "",
        "outFields": "*",
        "resultOffset": 0,
        "resultRecordCount": _max_record_count
    }
    # Get Count
    count = requests.get(layer_url+'/query', params={**gparams, 'returnCountOnly':True}, verify=False, headers=mod_headers).json()['count']
    print("-- Total Features: ", count)
    # Iterate
    for i in range(0, count, _max_record_count):
        print(f'-- Scrapping: Progress {(i/count):0.2f} %', end='\r')
        gparams["resultOffset"] = i
#         features = requests.get(layer_url+'/query', params=gparams, verify=False, headers=mod_headers).json()
#         feature_path = os.path.join(folderpath, f"{i}_{i+_max_record_count}.json")
#         print(feature_path)
#         with open(feature_path, 'w') as f:
#             json.dump(features, f)
    print(f'-- Scrapping: Progress 100 %', end='\n')

In [ ]:
for server_url in layers:
    r = requests.get(server_url, params=params, verify=False, headers=mod_headers)
    for lyr in r.json()['layers']:
        msg = f"\n-- Scrapping {lyr['name']}"
        lyr_url = server_url+'/'+str(lyr['id'])
        print(msg, lyr_url)
        #
#         metadata = requests.get(lyr_url, params=params, verify=False, headers=mod_headers).json()
        scrape(lyr_url)

C:\Users\san10428\AppData\Local\ESRI\conda\envs\dl\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mapservice.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,



-- Scrapping State https://mapservice.gov.in/mapserviceserv176/rest/services/Panchayat/panchayat_admin/MapServer/0


C:\Users\san10428\AppData\Local\ESRI\conda\envs\dl\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mapservice.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


-- Using max_record_count:  1


C:\Users\san10428\AppData\Local\ESRI\conda\envs\dl\lib\site-packages\urllib3\connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mapservice.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
